In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
from keras.models import load_model
from sklearn.metrics import f1_score

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
dir_path = '/content/drive/My Drive/DSCI552/Project/'

if not os.path.exists(dir_path):
    try:
        os.mkdir(dir_path)
        print(f'Created directory: {dir_path}')
    except OSError:
        print(f'Error creating directory: {dir_path}')

os.chdir(dir_path)

Mounted at /content/drive


In [ ]:
def model_prediction(test_dir):
    # categories
    categories = ['Pagodas','Pyramids','Modern','Gothic','Mughal','Neoclassical']

    # landmarks
    landmarks = [
          'TianningTemplePagoda','ThienMuPagoda','GiantWildGoosePagoda','ShwedagonPagoda',
          'FogongTemplePagoda','Pyramid of Giza','Pyramid of Djoser','Santa Cecilia Acatitlan Pyramid',
          'El Castillo, Chichen Itza','Louvre Pyramid','Chrysler Building','Cathedral of Brasília',
          'CCTV Headquarters','Hallgrímskirkja','eiffel','St.VitusCathedral',
          'MilanCathedral','ChartresCathedral','CologneCathedral','Notre-DameCathedral',
          'Taj Mahal','Tomb of Akbar','Jama Masjid','Tomb of I_timad-ud-Daulah',
          'Bibi Ka Maqbara','Buckingham Palace','Concertgebouw',
          'Academy of Athens','Panthéon','Ripon Building','other']

    #Collect data
    img_data_array, category_class, landmark_class, file_names = [], [], [], []
    valid_extensions = (".jpeg", ".jpg", ".JPG", ".webp", ".png")

    for category in os.listdir(test_dir):
        category_dir = os.path.join(test_dir, category)
        if not os.path.isdir(category_dir):
            continue

        for landmark in os.listdir(category_dir):
            landmark_dir = os.path.join(category_dir, landmark)
            if not os.path.isdir(landmark_dir):
                continue

            for file in os.listdir(landmark_dir):
                _, ext = os.path.splitext(file)
                if ext.lower() not in valid_extensions:
                    print(f"bad file type: {file}")
                    continue

                image_path = os.path.join(landmark_dir, file)
                image = cv2.imread(image_path)
                if image is None:
                    print(f"failed to load image: {image_path}")
                    continue

                image = cv2.resize(image, (224, 224), interpolation=cv2.INTER_AREA)
                image = np.array(image, dtype='float32') / 255
                img_data_array.append(image)
                category_class.append(category)
                landmark_class.append(landmark)
                file_names.append(file)

    #Prepare data
    X_test = np.array(img_data_array)
    
    #import pretrained models
    category_model = load_model('Models/category_model_f.h5')
    landmark_model = load_model('Models/landmark_model.h5')

    #predict categories
    category_probabilities = category_model.predict(X_test)
    category_predictions = np.argmax(category_probabilities, axis=1)
    category_predictions = [categories[i] for i in category_predictions]

    #predict landmarks
    cutoff_threshold = 0.3
    landmark_probabilities = landmark_model.predict(X_test)
    landmark_predictions = []
    for i in range(len(landmark_probabilities)):
      max_prob = np.max(landmark_probabilities[i])
      if max_prob >= cutoff_threshold:
        landmark_predictions.append(landmarks[np.argmax(landmark_probabilities[i])])
      else:
        landmark_predictions.append('other')

    #Format prediction dataframe
    prediction = pd.DataFrame({
        'image': file_names,
        'category': category_predictions,
        'category_confidence': np.max(category_probabilities, axis=1),
        'landmark': landmark_predictions,
        'landmark_confidence':np.max(landmark_probabilities, axis=1)
    })

    return prediction

In [ ]:
# os.listdir('/content/drive/My Drive/DSCI552/Project/')
model_prediction('/content/drive/My Drive/DSCI552/Project/Landmarks-v1_1/')

failed to load image: /content/drive/My Drive/DSCI552/Project/Landmarks-v1_1/Mughal/Bibi Ka Maqbara/Bibi Ka Maqbara - 14.jpg
failed to load image: /content/drive/My Drive/DSCI552/Project/Landmarks-v1_1/Neoclassical/Buckingham Palace/Buckingham Palace_19.jpg
failed to load image: /content/drive/My Drive/DSCI552/Project/Landmarks-v1_1/Neoclassical/Academy of Athens/Academy of Athens - 20.jpg
14/14 [==============================] - 238s 17s/step


,image,category,category_confidence,landmark,landmark_confidence
0,Bibi Ka Maqbara - 2.jpg,Modern,0.999946,ThienMuPagoda,0.945425
1,Bibi Ka Maqbara - 4.jpg,Modern,0.999567,ThienMuPagoda,0.859795
2,Bibi Ka Maqbara - 5.jpg,Modern,0.975614,ThienMuPagoda,0.550759
3,Bibi Ka Maqbara - 6.jpg,Modern,0.999205,ThienMuPagoda,0.913048
4,Bibi Ka Maqbara - 7.jpeg,Modern,0.999964,ThienMuPagoda,0.804958
...,...,...,...,...,...
412,Panth‚on_16.jpeg,Gothic,0.929090,ChartresCathedral,0.972167
413,Panth‚on_17.jpeg,Gothic,0.999585,ChartresCathedral,0.946539
414,Panth‚on_15.jpeg,Gothic,0.919845,ChartresCathedral,0.356729
415,Panth‚on_2.jpg,Gothic,0.999008,ChartresCathedral,0.870599
